In [ ]:
from osdatahub import FeaturesAPI, Extent, NGD
import geojson
import pandas as pd
import geopandas as gpd
import shapely.wkt
import shapely.geometry
import os

from key import ngd_key

pd.set_option("display.max_rows", 100)

data_dir = "../data/geo_analysis/os_licensing/"
os.makedirs(data_dir, exist_ok=True)

In [ ]:
def nrow(df):
    return print(f"No. of records in df: {len(df):,}")

## Data import

In [ ]:
# LPA boundary data from planning.data.gov

LAD_boundary_df = pd.read_csv("https://files.planning.data.gov.uk/dataset/local-authority-district.csv", 
                                  usecols = ["reference", "name", "geometry"])

# LAD_boundary_df.columns = ["geometry", "name", "LPACD"]


# load geometry and create GDF
LAD_boundary_df['geometry'] = LAD_boundary_df['geometry'].apply(shapely.wkt.loads)
LAD_boundary_gdf = gpd.GeoDataFrame(LAD_boundary_df, geometry='geometry')

# Transform to ESPG:27700 for more interpretable area units
LAD_boundary_gdf.set_crs(epsg=4326, inplace=True)
LAD_boundary_gdf.to_crs(epsg=27700, inplace=True)

nrow(LAD_boundary_gdf)
LAD_boundary_gdf.head()


In [ ]:
# get camden listed building data direct from endpoint
cmd_df = pd.read_csv("https://opendata.camden.gov.uk/api/views/uu3n-zgbj/rows.csv?accessType=DOWNLOAD")

# load geometry and create GDF
cmd_df['geometry'] = cmd_df['geometry'].apply(shapely.wkt.loads)
cmd_gdf = gpd.GeoDataFrame(cmd_df[["reference", "geometry"]], geometry='geometry')

cmd_gdf["geogcd"] = "E09000007"

# Transform to ESPG:27700 for more interpretable area units
cmd_gdf.set_crs(epsg=4326, inplace=True)
cmd_gdf.to_crs(epsg=27700, inplace=True)

nrow(cmd_gdf)
cmd_gdf.head()

In [ ]:
nwc_df = pd.read_csv("https://datamillnorth.org/download/listedbuildings/91298d09-45a6-4c6d-a909-11ec4eaab957/listedbuildings.csv")

# load geometry and create GDF
nwc_df['geometry'] = nwc_df['WKT'].apply(shapely.wkt.loads)
nwc_gdf = gpd.GeoDataFrame(nwc_df[["id", "geometry"]], geometry='geometry')

nwc_gdf["geogcd"] = "E08000021"

# Transform to ESPG:27700 for more interpretable area units
nwc_gdf.set_crs(epsg=27700, inplace=True)

nrow(nwc_gdf)
nwc_gdf.head()


In [ ]:
dnc_gdf = gpd.read_file("https://maps.doncaster.gov.uk/server/rest/services/Planning/Heritage_Map/MapServer/1/query?where=1%3D1&f=geojson&outFields=*")[["LBREFNO", "geometry"]]

dnc_gdf["geogcd"] = "E08000017"

dnc_gdf.set_crs(epsg=4326, inplace=True)
dnc_gdf.to_crs(epsg=27700, inplace=True)

nrow(dnc_gdf)
dnc_gdf.head()

In [ ]:
800/175000

In [ ]:
lpa_gdf = pd.concat([cmd_gdf, nwc_gdf, dnc_gdf])

lpa_gdf.groupby("geogcd").size()

In [ ]:
nrow(cmd_gdf)